# Getting Started with Google Search as a Tool with Gemini in Vertex AI

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb

### Set up the notebook

In [ ]:
# 1. Install Google Gen AI SDK for Python
# Install the following packages required to execute this notebook.

%pip install --upgrade --quiet google-genai

# Restart runtime
# To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

import IPython
​
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# -->⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️


# 2. Authenticate your Google Cloud account
# If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

import sys
​
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth
​
    auth.authenticate_user()

### Set Google Cloud project information and create client

In [ ]:
# Define project information
PROJECT_ID = "qwiklabs-gcp-00-3e05f4fe2c5f"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

# Create the API client
from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import (
    ApiKeyConfig,
    AuthConfig,
    EnterpriseWebSearch,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleMaps,
    GoogleSearch,
    LatLng,
    Part,
    Retrieval,
    RetrievalConfig,
    Tool,
    ToolConfig,
    VertexAISearch,
)

### Helper functions

def print_grounding_data(response: GenerateContentResponse) -> None:
    """Prints Gemini response with grounding citations in Markdown format."""
    if not (response.candidates and response.candidates[0].grounding_metadata):
        print("Response does not contain grounding metadata.")
        display(Markdown(response.text))
        return

    grounding_metadata = response.candidates[0].grounding_metadata
    markdown_parts = []

    # Citation indexes are in bytes
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)
    last_byte_index = 0

    for support in grounding_metadata.grounding_supports:
        markdown_parts.append(
            text_bytes[last_byte_index : support.segment.end_index].decode(ENCODING)
        )

        # Generate and append citation footnotes (e.g., "[1][2]")
        footnotes = "".join([f"[{i + 1}]" for i in support.grounding_chunk_indices])
        markdown_parts.append(f" {footnotes}")

        # Update index for the next segment
        last_byte_index = support.segment.end_index

    # Append any remaining text after the last citation
    if last_byte_index < len(text_bytes):
        markdown_parts.append(text_bytes[last_byte_index:].decode(ENCODING))

    markdown_parts.append("\n\n----\n## Grounding Sources\n")

    # Build Grounding Sources Section
    markdown_parts.append("### Grounding Chunks\n")
    for i, chunk in enumerate(grounding_metadata.grounding_chunks, start=1):
        context = chunk.web or chunk.retrieved_context
        if not context:
            continue

        uri = context.uri
        title = context.title or "Source"

        # Convert GCS URIs to public HTTPS URLs
        if uri and uri.startswith("gs://"):
            uri = uri.replace("gs://", "https://storage.googleapis.com/", 1).replace(
                " ", "%20"
            )
        markdown_parts.append(f"{i}. [{title}]({uri})\n")

    # Add Search/Retrieval Queries
    if grounding_metadata.web_search_queries:
        markdown_parts.append(
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_parts.append(
                f"\n**Search Entry Point:**\n{grounding_metadata.search_entry_point.rendered_content}\n"
            )
    elif grounding_metadata.retrieval_queries:
        markdown_parts.append(
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    display(Markdown("".join(markdown_parts)))

In [ ]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

### Example: Text generation without grounding

In [ ]:
PROMPT = "When is the next solar eclipse in the US?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

### Example: Text generation grounded in Google Search results

google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[google_search_tool]),
)

print_grounding_data(response)

### Example: Text generation with multimodal input grounded in Google Search results

In [ ]:
PROMPT = "What is the current temperature at this location?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://github-repo/generative-ai/gemini/grounding/paris.jpg",
            mime_type="image/jpeg",
        ),
        PROMPT,
    ],
    config=GenerateContentConfig(
        tools=[google_search_tool],
    ),
)

print_grounding_data(response)

### Example: Grounding with custom documents and data

In [ ]:
VERTEX_AI_SEARCH_PROJECT_ID = PROJECT_ID  # @param {type: "string"}
VERTEX_AI_SEARCH_REGION = "global"  # @param {type: "string"}
# Replace this with your App (Engine) ID from Vertex AI Search
VERTEX_AI_SEARCH_APP_ID = "my-vertex-ai-search-app_1749704883397"  # @param {type: "string"}

VERTEX_AI_SEARCH_ENGINE_NAME = f"projects/{VERTEX_AI_SEARCH_PROJECT_ID}/locations/{VERTEX_AI_SEARCH_REGION}/collections/default_collection/engines/{VERTEX_AI_SEARCH_APP_ID}"

PROMPT = "What is the Google company culture like?"

In [ ]:
# Text generation without grounding
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

# Text generation grounded in Vertex AI Search results
vertex_ai_search_tool = Tool(
    retrieval=Retrieval(
        vertex_ai_search=VertexAISearch(engine=VERTEX_AI_SEARCH_ENGINE_NAME)
    )
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the company culture like?",
    config=GenerateContentConfig(tools=[vertex_ai_search_tool]),
)

# print_grounding_data(response)
# print(response)
display(Markdown(response.text))

# Note that the response without grounding doesn't have any context about what company we are asking about. Whereas the response that was grounded in Vertex AI Search results contains information from the documents provided, along with citations of the information.

### Example: Grounded chat responses

You can also use grounding when using chat conversations in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

In [ ]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use?"

In [ ]:
# Now you can add the tools keyword arg with a Tool of GoogleSearch to instruct the chat model to first perform a Google Search with the prompt, then construct an answer based on the web search results:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(tools=[Tool(google_search=GoogleSearch())]),
)

display(Markdown("## Prompt"))
display(Markdown(f"> {PROMPT}"))
response = chat.send_message(PROMPT)
print_grounding_data(response)

display(Markdown("---\n"))

display(Markdown("## Follow-up Prompt"))
display(Markdown(f"> {PROMPT_FOLLOWUP}"))
response = chat.send_message(PROMPT_FOLLOWUP)
print_grounding_data(response)
